In [4]:
#dependencies
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
#from bayes_opt import BayesianOptimization

In [19]:
#Parameter values

#from Farquhar 1980
#turnover rate for carboxylation
kc = 2.5
#turnover rate for oxygenation
ko = .21*kc
#light intensity
I = 150
#total enzyme concentration
Et = 87.2
#CO2 concentration
C = 230
#O2 concentration
O = 210

#from Ye 2020
alpha = .295
beta = 2.42*(10**(-3))
gamma = 1.26*(10**(-4))

#couldnt find turnover # for RuBP so am setting kr, etc. = kc
kr = kc
kprod = kc
kE = kc

#also guesses here
kERC = .9*kc
kERO = .9*ko

kappac = 2/3
kappar = 1/3

In [24]:
#model for PGA reduction vs RuBP *with Ye model for light
def jprod(R, C = 230, O =210, kc = kc, ko = ko, kappac = kappac, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    jph = I*alpha*(1-beta*I)/(1+gamma*I)
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    jprod = ((1/kE)+(1/kappac*jph)+(1/jpga)-(1/(jpga+jph)))**(-1)
    return jprod

In [25]:
#plotting jprod against RuBP
@interact(Rsteps= (0, 5000, 10), Et = (0, 1000, 10), kr  = (.1, 10, .1))
def simulateR(C = 230, O =210, kc = kc, ko = ko, kappac=kappac, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = 10, kE = kE, kERO=kERO, kERC = kERC, Rsteps=300):
    vcr = []
    for k in range(1,Rsteps):
        vcr.append(jprod(k, C=C, O=O, kc = kc, ko = ko, kappac=kappac, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC))
    ksum = ko*O/kERO + kc*C/kERC
    
    plt.plot(range(1,Rsteps), vcr)
    plt.xlabel('RuBP')
    plt.ylabel('jprod')
    plt.show()

interactive(children=(IntSlider(value=230, description='C', max=690, min=-230), IntSlider(value=210, descripti…

In [28]:
#plotting jprod against irradiance, still with Ye model
@interact(Isteps= (0, 5000, 10), Et = (0, 1000, 10), kr  = (.1, 10, .1))
def simulateI(R= 100, C = 230, O =210, kc = kc, ko = ko, kappac=kappac, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = 10, kE = kE, kERO=kERO, kERC = kERC, Isteps=1000):
    vcI = []
    for i in range(1,Isteps):
        vcI.append(jprod(R=R, C=C, O=O, kc = kc, ko = ko, kappac=kappac, I = i, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC))
    
    plt.plot(range(1,Isteps), vcI)
    plt.xlabel('Irradiance')
    plt.ylabel('jprod')
    plt.show()

interactive(children=(IntSlider(value=100, description='R', max=300, min=-100), IntSlider(value=230, descripti…

In [8]:
#various models for irradiance
#parameters
zun = .21
zdo = .33

In [9]:
#Ye model
def Ye(I, alpha = alpha, beta = beta, gamma = gamma):
    jph = alpha*(1-beta*I)*I/(1+gamma*I)
    return jph

In [10]:
#Ye model with irradiance dependance
def altYe(I, alpha = alpha, beta = beta, zun = zun, zdo = zdo, gamma = gamma):
    frac = beta*I/(zun*I+zdo)
    jph = I*alpha*(1-frac)/(1+gamma*I)
    return jph

In [11]:
#circuit formulation
Imax = 700
Rc = .01
def circuit(I, Imax=Imax, Rc=Rc):
    jph = I/((I/Imax)+Rc)
    return jph

In [12]:
#Various models as functions of irradiance
@interact()
def simulatejph(irange=5000, zun=zun, zdo=zdo, Imax = Imax, Rc=Rc, alpha = alpha, beta = beta, gamma = gamma):
    jphylist = []
    jphalist = []
    jphblist = []
    for i in range(1,irange):
        jphylist.append(Ye(i, alpha = alpha, beta = beta, gamma = gamma))
        jphalist.append(altYe(i, alpha = alpha, beta = beta, zun = zun, zdo = zdo, gamma = gamma))
        jphblist.append(circuit(i, Imax, Rc))
    plt.plot(range(1,irange), jphylist, label='Ye model')
    plt.plot(range(1, irange), jphalist, label = 'model A')
    plt.plot(range(1,irange), jphblist, label = 'model B')
    plt.xlabel('Irradiance')
    plt.ylabel('jph')
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=5000, description='irange', max=15000, min=-5000), FloatSlider(value=0.2…

In [33]:
#RuBP regeneration
alphaa = 1
alphab = 1
def dRdtalt(R, C = 230, O =210, kc = kc, ko = ko, kappac = kappac, kappar = kappar, alphaa = alphaa, alphab = alphab, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    jph = I*alpha*(1-beta*I)/(1+gamma*I)
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    kEa = alphaa*kc
    kEb = alphab*kc
    jpc = kappac*jph
    jpr = kappar*jph
    jprod = (5/6)*((1/kEa)+(1/(kappac*jpc))+(1/jpga)-(1/(jpga+jpc)))**(-1)
    jrubp = (1/kEb + 1/(kappar*jpr) + 1/jprod - 1/(jpr+jprod))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + jrubp
    return dRdt

In [34]:
@interact(alpha = (0,100,.1))
#note that R does not scale to realistic values, likely due to wrong parameterization
def simulatealtR(Ro = .0014, tsteps = 500, dt = .001, kappac = kappac, kappar = kappar, C = 230, O =210, kc = kc, ko = ko, alphaa = alphaa, alphab=alphab, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    R = np.zeros(tsteps)
    R[0] = Ro
    for t in range(tsteps-1):
        R[t+1] = R[t] + dRdtalt(R[t], C=C, alpha = alpha, alphaa = alphaa, alphab = alphab)*dt
    plt.plot(range(tsteps),R, label = 'RuBP')
    plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(FloatSlider(value=0.0014, description='Ro', max=0.0042, min=-0.0014), IntSlider(value=50…

In [48]:
#params
kappaa = 4
m = .2
#these params are made up. Might need high m for NPQ to make a difference
kf = .2
knpq = .1
kpq = .7

In [84]:

#Alternative light model: dynamical system with balancing fluxes
def jprodalt(pq, I, R):
    jph = pq
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    kEa = alphaa*kc
    kEb = alphab*kc
    jpc = kappac*jph
    jpr = kappar*jph
    jprod = ((1/kEa)+(1/(kappac*jpc))+(1/jpga)-(1/(jpga+kappac*jpc)))**(-1)
    jprodr = (5/6)*jprod
    jrubp = (1/kEb + 1/(kappar*jpr) + 1/jprodr - 1/(kappar*jpr+jprodr))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + jrubp
    return jprod, dRdt, jpga

In [97]:
def jpq(pq, I, R, jf=.1*I, m=.2, jsyn=1.5):
    jp, jr, jpga = jprodalt(pq, I, R)
    jadp = jp + jr
    jatp = (1/jsyn + 1/(kappaa*pq) + 1/jadp - 1/(kappaa*pq + jadp))**(-1)
    jnpq = m*(pq - jatp)
    jpq = I*kpq/(kpq+kf+knpq*jnpq)
    return jp, jpq, jr, jnpq, jpga, jatp, jadp

In [98]:
#light input
def logistic(Io, Imax, x, kl):
    f = Imax/(1+np.exp(-kl*(x-Io)))
    return f

In [101]:
@interact(pqo = (0,100,.1), Ro=(0.001,.002,.0001), time = (0,10,1))
def integrated(pqo, Ro, m=m, ik = 5000, Io=100, Imax=500, kl = 1, time=1000, dt = .001, C = 230, O =210, kc = kc, ko = ko, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    tsteps = np.linspace(0, time, time*10)
    jplist = np.zeros(len(tsteps))
    jpqlist = np.zeros(len(tsteps))
    Rlist = np.zeros(len(tsteps))
    npqlist = np.zeros(len(tsteps))
    jpgalist = np.zeros(len(tsteps))
    atplist = np.zeros(len(tsteps))
    adplist = np.zeros(len(tsteps))
    jpqlist[0] = pqo
    Rlist[0] = Ro
    jplist[0] = jprodalt(pqo, ik, Ro)[0]
    jpgalist[0] = jprodalt(pqo, ik, Ro)[2]
    ilist = [ik for t in tsteps]
    for t in range(len(tsteps)-1):
        jp, jpqu, jr, jnpq, jpga, jatp, jadp = jpq(jpqlist[t], ilist[t], Rlist[t], m=m)
        Rlist[t+1] = Rlist[t] + jr*dt
        jplist[t+1] = jp
        jpqlist[t+1] = jpqu
        atplist[t+1] = jatp
        adplist[t+1] = jadp
        npqlist[t] = jnpq
        jpgalist[t] = jpga
    atplist[0] = atplist[1]
    adplist[0] = adplist[1]
    fig, axs = plt.subplots(3)
    axs[0].plot(tsteps, jpqlist, label='Jpq')
    axs[1].plot(tsteps, Rlist, label = 'RuBP', color='grey')
    axs[0].plot(tsteps, ilist, label='Irradiance')
    axs[1].plot(tsteps, jplist, label = 'jprod', color='red')
    jpgalist[-1] = jpgalist[-2]
    axs[1].plot(tsteps, jpgalist, label = 'jpga')
    npqlist[-1] = npqlist[-2]
    axs[0].plot(tsteps, npqlist, label='NPQ')
    axs[2].plot(tsteps, atplist, label='jATP', color='brown')
    axs[2].plot(tsteps, adplist, label='jADP', color='pink')
    axs[2].legend()
    axs[0].legend(loc='upper right')
    axs[1].legend(loc='upper right')  
    axs[0].set(xlabel='time')
    axs[1].set(xlabel='time')
    axs[2].set(xlabel='time')

interactive(children=(FloatSlider(value=50.0, description='pqo'), FloatSlider(value=0.0015, description='Ro', …